#### Prerequesites:
* Define schema (make figure) 
* Define structure (make figure)
* Define schema class in python (src/models/model_schemata.py)
* Convert schema + structure into model template (models/templates/p_model.pl)

#### Procedure:
* Load data
* Adjust model script for lfi accordingly
* Create evidence file (if I don't need different evidence files, move to builmodel notebook)
* LFI

Multiple instances, 1 pst, X enzymes, one sample per instance, multiple enzymes per instance, test how many enzymes can have in one instances, how computing time depends on N p::f for 60 samples

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import random

from src.models import model_schemata as schema
from src.models import build_model as build
from src.visualization import visualize as viz

In [3]:
# os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

#### Training/Testing

In [4]:
# reading csv from files into dict
data = {}
data['e_ksea'] = pd.read_csv('data/processed/ebdt_data/sub_network_p/e_ksea.csv')
data['p_fc'] = pd.read_csv('data/processed/ebdt_data/sub_network_p/p_fc.csv')
data['es_interaction'] = pd.read_csv('data/processed/ebdt_data/sub_network_p/es_interaction.csv')
data['e_ksea']

,enzyme,sample,tc,value,prob,p_dec,p_inc
0,ABL1,AC220,10.0,inc,0.649536,0.001000,0.649536
1,ABL1,AT13148,10.0,inc,0.591795,0.001000,0.591795
2,ABL1,AZ20,10.0,inc,0.652697,0.001000,0.652697
3,ABL1,AZD1480,10.0,inc,0.570660,0.001000,0.570660
4,ABL1,AZD3759,10.0,dec,0.837416,0.837416,0.001000
...,...,...,...,...,...,...,...
483,SRC,Torin,12.0,inc,0.775683,0.001000,0.775683
484,SRC,Trametinib,12.0,inc,0.736235,0.001000,0.736235
485,SRC,U73122,12.0,inc,0.551325,0.001000,0.551325
486,SRC,Ulixertinib,12.0,inc,0.602812,0.001000,0.602812


Splitting data into training/testing

In [10]:
# getting sample names (union of all samples in all datasets)
samples = list(set(data['e_ksea']['sample']).union(set(data['p_fc']['sample'])))
samples.sort()
# sample x% of samples randomly without replacement with seed
random.seed(612)
train = random.sample(samples, int(len(samples)*1))
print(train)
# filter data
training_data = {}
training_data['e_ksea'] = data['e_ksea'][data['e_ksea']['sample'].isin(train)].reset_index(drop=True)
training_data['p_fc'] = data['p_fc'][data['p_fc']['sample'].isin(train)].reset_index(drop=True)

['TAK715', 'TGX221', 'CHIR99021', 'CAL101', 'Ripasudil', 'KN93', 'LY2584702', 'LY2090314', 'Ku0063794', 'SP600125', 'Ribociclib', 'U73122', 'AZD5438', 'GDC0994', 'Edelfosine', 'PF3758309', 'Amuvatinib', 'AT13148', 'AZD3759', 'Go6976', 'JNJ', 'NU7441', 'GSK2334470', 'Ipatasertib', 'AZD5363', 'LY2835219', 'Torin', 'Bosutinib', 'H89', 'DNAPK', 'GDC0941', 'FRAX486', 'CX4945', 'HS173', 'Linsitinib', 'Dabrafenib', 'AZD1480', 'BX912', 'AZ20', 'TBCA', 'Selumetinib', 'GF109203X', 'AZD6482', 'AC220', 'PH797804', 'AZD8055', 'Trametinib', 'JNK']


In [11]:
# Mapping data to Problog predicates
predicates = {}
predicates['e_ksea'] = schema.EKseaPredicate()
predicates['e_ksea'].add_data(training_data['e_ksea'], 'enzyme', 'sample', 'value')
predicates['p_fc'] = schema.PFoldChangePredicate()
predicates['p_fc'].add_data(training_data['p_fc'], 'phosphosite', 'sample', 'value')

In [12]:
samples = list(set(training_data['e_ksea']['sample']).union(set(training_data['p_fc']['sample'])))
phosphosites = list(set(training_data['p_fc']['phosphosite']))
evidence_dict = {}
for s in samples:
    evidence_dict[s] = {}
    for p in phosphosites:
        enzymes = data['es_interaction']['enzyme'][data['es_interaction']['phosphosite'] == p].tolist()
        evid_generator = build.ProblogStatementGenerator(predicates['e_ksea'])
        evidence_e = []
        for e in enzymes:
            evidence_e.extend(evid_generator.generate_facts(build.EvidenceTemplate, select=[s, e]))
        evid_generator = build.ProblogStatementGenerator(predicates['p_fc'])
        evidence_p = evid_generator.generate_facts(build.EvidenceTemplate, select=[s, p])
        evidence_dict[s][p] = evidence_e + evidence_p

In [9]:
# write evidence_dict to text file: loop over samples and separate by line of hyphens
with open('models/ebdt_data/sub_network/p_model/evidence_exp9.pl', 'w') as f:
    for s in evidence_dict:
        for p in evidence_dict[s]:
            for e in evidence_dict[s][p]:
                f.write(e + '\n')
            f.write('--------------------' + '\n')

In [13]:
# write evidence_dict to text file: loop over samples and separate by line of hyphens
with open('models/ebdt_data/sub_network/p_model/evidence_exp9_training.pl', 'w') as f:
    for s in evidence_dict:
        for p in evidence_dict[s]:
            for e in evidence_dict[s][p]:
                f.write(e + '\n')
            f.write('--------------------' + '\n')

LFI

In [15]:
DIR = 'models/ebdt_data/sub_network/p_model/'
max_iter = 200
# learning from interpretation
cmd = f'problog lfi {DIR}p_model_lfi_exp9.pl {DIR}evidence_exp9.pl -O {DIR}p_model_exp9_200i.pl --logger {DIR}log_exp9-200i.txt -k ddnnf -v -n {max_iter}'
os.system(cmd)

-5828.5768921144 [0.00022445, 0.04111104, 0.00139734, 0.88208698, 0.00094823, 0.71466641, 0.0255433, 0.00238325, 0.0059561, 0.0064628, 0.63375229, 0.94178802, 0.76483695, 0.90384655, 0.34788978, 0.00346999, 0.00834978, 0.00852122, 0.65512851, 0.00224042, 0.97918787, 0.0019618, 0.01367958, 0.76973641, 0.65805187, 0.5942344, 0.76330317, 0.80526139, 0.00089754, 0.0007473, 0.10247135, 0.68043185, 0.99297228, 0.07477701, 0.96482117, 0.00136982, 0.73989235, 0.00148483, 0.00100794, 0.89245372, 0.78574793, 0.00120872, 0.00166663, 0.8914393, 0.00086295, 0.00074114, 0.82916155, 0.81799386, 0.01343053, 0.01002336, 0.00087749, 0.00171373, 0.62582718, 0.01618534, 0.00095918, 0.00225043, 0.00020156, 0.01188109, 0.15184029, 0.00167876, 0.90932389, 0.00169074, 0.00086113, 0.13429303, 0.95859789, 0.72466295, 0.00147573, 0.78929858, 0.00098154, 0.00073245, 0.03070453, 0.49319228, 0.51192536, 0.00070821, 0.73594476, 0.00089341, 0.00093495, 0.0002943, 0.00174016, 0.00302396, 0.9896325, 0.06911937, 0.00176

0

In [13]:
DIR = 'models/ebdt_data/sub_network/p_model/'
# learning from interpretation
cmd = f'problog ground {DIR}p_model_lfi_exp9.pl -o {DIR}p_model_exp9_ground.pl --format pl'
os.system(cmd)

(True, 't(0.33,"SRC","KN93")::e_activity("SRC","KN93",inc); t(0.33,"SRC","KN93")::e_activity("SRC","KN93",dec); t(0.33,"SRC","KN93")::e_activity("SRC","KN93",base).\nt(0.33,"ABL1","KN93")::e_activity("ABL1","KN93",inc); t(0.33,"ABL1","KN93")::e_activity("ABL1","KN93",dec); t(0.33,"ABL1","KN93")::e_activity("ABL1","KN93",base).\nt(0.33,"PTK2","KN93")::e_activity("PTK2","KN93",dec); t(0.33,"PTK2","KN93")::e_activity("PTK2","KN93",base); t(0.33,"PTK2","KN93")::e_activity("PTK2","KN93",inc).\nt(0.33,"PTPRG","KN93")::e_activity("PTPRG","KN93",inc); t(0.33,"PTPRG","KN93")::e_activity("PTPRG","KN93",base); t(0.33,"PTPRG","KN93")::e_activity("PTPRG","KN93",dec).\nt(0.33,"FYN","KN93")::e_activity("FYN","KN93",inc); t(0.33,"FYN","KN93")::e_activity("FYN","KN93",dec); t(0.33,"FYN","KN93")::e_activity("FYN","KN93",base).\nt(0.33,"PTK6","KN93")::e_activity("PTK6","KN93",dec); t(0.33,"PTK6","KN93")::e_activity("PTK6","KN93",base); t(0.33,"PTK6","KN93")::e_activity("PTK6","KN93",inc).\nt(0.5,"SRC","P

256